In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
from trl import SFTTrainer
import torch
from transformers import BitsAndBytesConfig, TrainingArguments

cache_dir="E:\Cache\Hugging_Face"

comet_ml is installed but `COMET_API_KEY` is not set.


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "hfl/llama-3-chinese-8b-instruct"

# The instruction dataset to use

dataset_name='CocoNutZENG/Epilepsy_Synthetics'

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 8

# Alpha parameter for LoRA scaling
lora_alpha = 8

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "../../Data/checkpoints"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 1e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 10

log_dir='../../Data/logs'

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = "auto"


################################################################################
# Model save
################################################################################
model_save_dir='../../Data/saves'

In [3]:
import re
import unicodedata

template_wInput_en = """Below is an instruction that describes a task. \
Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
template_no_input_en = """Below is an instruction that describes a task. \
Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""
template_wInput_zh = """下面是一条任务的指令。 \
请按照所给出指令和输入生成适当输出。

### 指令：
{instruction}

### 输入：
{input}

### 回复：
"""
template_no_input_zh = """下面是一条任务的指令。 \
请按照所给出指令生成适当输出。

### 指令：
{instruction}

### 回复：
"""
def detect_character_language(string):
    first_char = string[0:3]
    
    if re.match(r'[A-Za-z]', first_char):
        return True
    else:
        return False
def data_to_string(data):

    instruction = data["Instruction"]
    context = data["Input"]
    response = data["Output"]
    if (detect_character_language(instruction)):
        template = template_wInput_en if len(context) > 0 else template_no_input_en
    else:
        template = template_wInput_zh if len(context) > 0 else template_no_input_zh
        
    source = template.format(instruction=instruction, input=context)

    return {
        "source": source,
        "text": source + response,
    }

dataset=load_dataset(dataset_name,cache_dir=cache_dir)
train_data=dataset['train'].map(data_to_string).remove_columns(['Instruction', 'Input', 'Output'])
valid_data=dataset['validation'].map(data_to_string).remove_columns(['Instruction', 'Input', 'Output'])


In [4]:
IGNORE_TOKEN = -100

def data_collator(features, tokenizer):
    sources = [feature["source"] for feature in features]
    targets = [feature["text"] for feature in features]

    source_tokens = tokenizer(
        sources,
        return_tensors="pt",
        padding='longest',
        max_length=None,
    )

    target_tokens = tokenizer(
        targets,
        return_tensors="pt",
        padding='longest',
        max_length=None,
    )

    labels = target_tokens["input_ids"].clone()
    
    

    for i in range(len(labels)):
        source_len = source_tokens["attention_mask"][i].sum()

        labels[i, :source_len] = IGNORE_TOKEN

    res = {
        "input_ids": target_tokens["input_ids"],
        "attention_mask": target_tokens["attention_mask"],
        "labels": labels,
    }
    return res

In [5]:

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    cache_dir=cache_dir
)
model.config.use_cache = True
model.config.pretraining_tp = 1



# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,cache_dir=cache_dir,attn_implementation='sdqa',)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules='all-linear',
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


Your GPU supports bfloat16: accelerate training with bf16=True


E:\anaconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [7]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,

    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    logging_dir=log_dir,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    neftune_noise_alpha=5,
)


Map:   0%|          | 0/1624 [00:00<?, ? examples/s]

In [8]:
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    trainer.train()


E:\anaconda3\envs\DL\lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
20,2.218700
40,1.614800
60,1.102900
80,0.932700
100,0.711300
120,1.052800
140,0.729800
160,0.750900
180,0.852900
200,0.645200


TrainOutput(global_step=1624, training_loss=0.6455535027634334, metrics={'train_runtime': 10364.8361, 'train_samples_per_second': 0.313, 'train_steps_per_second': 0.157, 'total_flos': 1.3749541410521088e+16, 'train_loss': 0.6455535027634334, 'epoch': 2.0})

In [9]:
trainer.evaluate()
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

('../../Data/saves\\tokenizer_config.json',
 '../../Data/saves\\special_tokens_map.json',
 '../../Data/saves\\tokenizer.json')

In [11]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CocoNutZENG/checkpoints/commit/3060d28d01ae502f27b0f4f19757ad228771ece6', commit_message='End of training', commit_description='', oid='3060d28d01ae502f27b0f4f19757ad228771ece6', pr_url=None, pr_revision=None, pr_num=None)